# Google Play Store Apps Data
Esse conjunto de dados consiste em dados retirados da internet de mais de 10000 aplicativos da Google Play Store.

[Fonte](https://www.kaggle.com/lava18/google-play-store-apps) do conjunto de dados

### _Esse projeto está em construção. Disponibilizei para já verem o progresso!_ 
### _Agradeço a visita! :)_
    _Última alteração feita em 31/08/2023_

    Utilizarei SQL para análises e Power BI para visualizações
    
O arquivo 'apps_data.csv' foi inserido na tabela 'Apps_data' para limpeza e tratamento dos dados.

Também utilizei a tabela 'Apps_test' com os mesmos valores, para que seja possível comparar o antes e depois desse processo.

# Limpeza e tratamendo dos dados - SQL

## Coluna 'App' - Valores duplicados

Dicionário: coluna 'App' contém o nome do aplicativo

A seguir podemos ver que existem 10841 registros na tabela, sendo 9660 valores únicos e 1181 valores duplicados.

![dup_qtdade](./img/dup_qtdade.PNG)

teste png sem ./

![dup_qtdade](img/dup_qtdade.PNG)

teste attachment img

![dup_qtdade](attachment: img/dup_qtdade.PNG)

teste attachment

![dup_qtdade](attachment: dup_qtdade.PNG)

teste png

![dup_qtdade](./img/dup_qtdade.PNG)

teste img src="./img

<img src="./img/dup_qtdade.PNG">

teste img />

<img src="./img/dup_qtdade.PNG"/>

teste img width

<img src="./img/dup_qtdade.PNG" width="800" />


**COMANDO USADO ACIMA:**
`SELECT COUNT(app) AS TOTAL,`
       `COUNT(DISTINCT app) AS ÚNICOS,` 
       `(COUNT(app) - COUNT(DISTINCT app)) AS VALORES_DUPLICADOS`
`FROM apps_data`

Foi realizado o script SQL abaixo para verificar os registros duplicados.

![dup1](./img/dup1.PNG)

**COMANDO USADO ACIMA:**
`SELECT * FROM (`
`SELECT *, `
`ROW_NUMBER() 	OVER(`
		`PARTITION BY app`
		`ORDER BY (app)`
		`) AS POSICAO`
`FROM apps_data`
		`) TABELA`
`WHERE POSICAO > 1`
`ORDER BY app`

Para deletar os valores duplicados, foi executado o comando SQL abaixo:

![dup_delete](img/dup_delete.PNG)

**COMANDO USADO ACIMA:**
`DELETE FROM apps_data WHERE id IN (`
`SELECT id FROM (`
`SELECT *, `
`ROW_NUMBER() 	OVER(`
		`PARTITION BY app`
		`ORDER BY (app)`
		`) AS POSICAO`
`FROM apps_data`
		`) TABELA`
`WHERE POSICAO > 1)`

Após a aplicação desse DELETE, podemos ver que não existem mais valores duplicados

![dup_now](img/dup_now.PNG)

**COMANDO USADO ACIMA:**
`SELECT COUNT(app) AS TOTAL,`
       `COUNT(DISTINCT app) AS ÚNICOS,` 
       `(COUNT(app) - COUNT(DISTINCT app)) AS VALORES_DUPLICADOS`
`FROM apps_data`

## Coluna 'Category'

Dicionário: coluna 'Category' contém a categoria que o aplicativo pertence

Verificando se existem valores faltando e julgar se é necessário realizar a limpeza

Podemos ver que existem 34 categorias, sendo que somente 1 registro não tem valor nessa coluna (caracterizado pelo valor NULL)

![category_distinct](img/category_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT category, COUNT(*)`
`FROM apps_data`
`GROUP BY category`

Abaixo vemos um pouco dos detalhes do único registro com valor NULL

![category_null](img/category_null.PNG)

**COMANDO USADO ACIMA:**
`SELECT *`
`FROM apps_data`
`WHERE category IS NULL`

Como esta coluna é do tipo VARCHAR (texto), o valor NULL não precisa ser corrigido, já que não temos informações suficientes para atribuir um valor.


## Coluna 'Rating'

Dicionário: coluna 'Rating' contém a avaliação que o usário deu para o aplicativo (varia de 1.0 a 5.0)

Ao executar o SQL abaixo, podemos ver que existe valores 'NaN', ou seja, valores faltando. 

![rating_old](img/rating_old.PNG)

**COMANDO USADO ACIMA:**
`SELECT MIN(rating), MAX(rating)`
`FROM apps_data`

A seguir vemos que existem 1463 registros com o valor 'NaN'

![rating_NaN_1463](img/rating_NaN_1463.PNG)

**COMANDO USADO ACIMA:**
`SELECT rating, COUNT(*)`
`FROM apps_data`
`WHERE rating = 'NaN'`
`GROUP BY rating`

Como a variação é de 1.0 a 5.0, foi atribuído o valor 0 para os campos com 'NaN'. Atribuí um valor númerico para que no futuro possamos realizar operações numéricas com essa coluna.

Por exemplo: a função AVG() - média, não pode ser executada com campos do tipo texto:

![rating_error_avg_text](img/rating_error_avg_text.PNG)

**COMANDO USADO ACIMA:**
`SELECT MIN(rating), MAX(rating), AVG(rating)`
`FROM apps_data`

Executei o UPDATE para atribuir o valor 0 no lugar de 'NaN'

![rating_update](img/rating_update.PNG)

**COMANDO USADO ACIMA:**
`UPDATE apps_data`
`SET rating = 0`
`WHERE rating = 'NaN'`

Ainda assim, o tipo da coluna permanece como texto, então converti a coluna para o tipo numérico

![rating_alter_type](img/rating_alter_type.PNG)

**COMANDO USADO ACIMA:**
`ALTER TABLE apps_data`
`ALTEER COLUMN rating TYPE NUMERIC`
`USING rating::NUMERIC`

Podemos ver a seguir que a funçao AVG() já funciona normalmente. 

Obs: excluímos o valor 0 da consulta, uma vez que o 0 foi atribuído por mim e não reflete a nota real do aplicativo

![rating_min_max__numeric](img/rating_min_max__numeric.PNG)

**COMANDO USADO ACIMA:**
`SELECT MIN(rating), MAX(rating), AVG(rating)`
`FROM apps_data`
`WHERE rating <> 0`

## Coluna 'Reviews'

Dicionário: coluna 'Reviews' contém o número de avaliações do aplicativo

Abaixo vemos que essa coluna não possui valores faltando

![reviews_zero_null](img/reviews_zero_null.PNG)

**COMANDO USADO ACIMA:**
`SELECT * `
`FROM apps_data`
`WHERE reviews IS NULL`

Quando executamos o script abaixo, vemos que ele retorna o valor 9992 como máximo, mas isso acontece porque essa coluna foi atribuída o tipo TEXTO, então ele organiza por 'ordem alfabética'.

![reviews_min_max](img/reviews_min_max.PNG)

**COMANDO USADO ACIMA:**
`SELECT MIN(reviews), MAX(reviews)`
`FROM apps_data`

Executei o script de UPDATE para alterar o tipo para INT

![reviews_alter_type](img/reviews_alter_type.PNG)

**COMANDO USADO ACIMA:**
`ALTER TABLE apps_data`
`ALTER COLUMN reviews TYPE INT`
`USING reviews::INT`

Agora vemos o máximo com o valor que realmente é o maior da coluna

![review_int_type](img/review_int_type.PNG)

**COMANDO USADO ACIMA:**
`SELECT MIN(reviews), MAX(reviews)`
`FROM apps_data`

## Coluna 'Size'

Dicionário: coluna 'Size' contém a o tamanho do aplicativo

Como podemos ver, existem 3 valores para essa coluna. 

Valores que terminam em 'K' (Kilobytes), em 'M' (Megabytes) e 'Varies with device' (varia com o dispositivo)

![size_distinct](img/size_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT size, COUNT(*)`
`FROM apps_data`
`GROUP BY size`

Podemos ver que não há valores NULL ou '0', portanto não há valores faltando

![size_not_null](img/size_not_null.PNG)

**COMANDO USADO ACIMA:**
`SELECT * `
`FROM apps_data`
`WHERE size IS NULL`

![size_zero](img/size_zero.PNG)

**COMANDO USADO ACIMA:**
`SELECT * `
`FROM apps_data`
`WHERE size = '0'`

Como se trata de uma coluna que podemos realizar operações como média do tamanho dos aplicativos. Foi necessária alguns tratamentos para transformar os dados em números.

Abaixo vemos que não existem valores do tipo 'B' (Bytes)

![size_count_B](img/size_count_B.PNG)

**COMANDO USADO ACIMA:**
`SELECT COUNT(*)`
`FROM apps_data`
`WHERE UPPER(size) LIKE '%B%'`

Vemos que existem 316 valores do tipo 'K' (Kilobytes)

![size_count_K](img/size_count_K.PNG)

**COMANDO USADO ACIMA:**
`SELECT COUNT(*)`
`FROM apps_data`
`WHERE UPPER(size) LIKE '%K%'`

Vemos que existem 8830 valores do tipo 'M' (Megabytes)

![size_count_M](img/size_count_M.PNG)

**COMANDO USADO ACIMA:**
`SELECT COUNT(*)`
`FROM apps_data`
`WHERE UPPER(size) LIKE '%M%'`

E existem 1695 valores do tipo 'Varies with device'

![size_count_with](img/size_count_with.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT size, COUNT(*)`
`FROM apps_data`
`WHERE LOWER(size) LIKE '%with%'`
`GROUP BY size`

Totalizando os 10841 valores que a tabela original possuía.

Para transformar os dados em números foi realizado o comando para excluir o 'K' dos valores e ficar somente os números

![size_update_K](img/size_update_K.PNG)

**COMANDO USADO ACIMA:**
`UPDATE apps_data`
`SET size = (RTRIM(UPPER(size), 'K')::NUMERIC)`
`WHERE UPPER(size) LIKE '%K%'`

Para transformar os dados em números foi realizado o comando para excluir o 'M' dos valores e ficar somente os números multiplicados por 1000 (cada 1M corresponde a 1000K, aproximadamente)

![size_update_M](img/size_update_M.PNG)

**COMANDO USADO ACIMA:**
`UPDATE apps_data`
`SET size = ((RTRIM(UPPER(size), 'K')::NUMERIC) * 1000)`
`WHERE UPPER(size) LIKE '%M%'`

E os valores 'Varies with device' foram substituídos por '0' para que possamos executar operações matemáticas com essa coluna. Sendo que não existia nos dados originais o valor '0', por isso escolhi ele

![size_update_WITH](img/size_update_WITH.PNG)

**COMANDO USADO ACIMA:**
`UPDATE apps_data`
`SET size = '0'`
`WHERE UPPER(size) LIKE '%WITH%'`

Transformei o tipo da coluna para valores inteiros (INT)

![size_update_type](img/size_update_type.PNG)

**COMANDO USADO ACIMA:**
`ALTER TABLE apps_data`
`ALTER COLUMN size TYPE NUMERIC`
`USING size::NUMERIC`

Podendo assim realizar operações matemáticas com os valores. Excluindo o valor 0 já que ele foi adicionado para classificar os valores 'Varies with device'.

![size_min_max__numeric](img/size_min_max__numeric.PNG)

**COMANDO USADO ACIMA:**
`SELECT MIN(size), MAX(size), AVG(size)`
`FROM apps_data`
`WHERE size <> 0`

Lembrando que agora os valores estão na ordem de 'K' (Kilobytes), ou seja, uma coluna com valor 250 corresponde a 250 Kilobytes; e uma coluna com valor 1000 corresponde a 1000 Kilobytes ou 1 Megabytes.

## Coluna 'Installs'

Dicionário: coluna 'Installs' contém o número de vezes que o aplicativo foi instalado

Essa coluna não possui valores faltantes, mas sim 21 valores distintos

![installs_distinct](img/installs_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT installs`
`FROM apps_data`
`ORDER BY installs DESC`

Por se tratar de uma coluna que representa o número de vezes que o aplicativo foi instalado, todos os número possuem o símbolo '+' para informar que são mais valores do que representado pelo número.

![installs_old](img/installs_old.PNG)

**COMANDO USADO ACIMA:**
`SELECT app, installs`
`FROM apps_data`

Porém como eu gostaria de realizar operações matemáticas com esses números, primeiro tive que retirar a vírgula que separava os números

![installs_update_virgula](img/installs_update_virgula.PNG)

**COMANDO USADO ACIMA:**
`UPDATE apps_data`
`SET installs = REPLACE(installs, ',', '')`
`WHERE installs LIKE '%,%'`

Depois tive que retirar o símbolo '+', para restar somente os números

![installs_update_plus](img/installs_update_plus.PNG)

**COMANDO USADO ACIMA:**
`UPDATE apps_data`
`SET installs = RTRIM(installs, '+')::NUMERIC`
`WHERE installs LIKE '%+%'`

E então converter a coluna do tipo VARCHAR para o tipo numérico

![installs_alter_type](img/installs_alter_type.PNG)

**COMANDO USADO ACIMA:**
`ALTER TABLE apps_data`
`ALTER COLUMN installs TYPE NUMERIC`
`USING installs::NUMERIC`

Resultando numa coluna onde é possível efetuar operações matemáticas, conforme vemos a seguir

![installs_min_max](img/installs_min_max.PNG)

**COMANDO USADO ACIMA:**
`SELECT MIN(installs), MAX(installs)`
`FROM apps_data`

Restou 20 registros, já que o 0+ e o 0 se uniram

![installs_new_distinct](img/installs_new_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT installs`
`FROM apps_data`
`ORDER BY installs DESC`

## Coluna 'Type'

Dicionário: coluna 'Type' contém qual é o tipo do aplicativo. Podendo ser de 2 tipos: Grátis (Free) e Pago (Paid)

Como 'Type' é do tipo VARCHAR, não foi necessário tratar o único valor faltante representado por 'NaN'.

Podemos ver que são 8903 aplicativos grátis (Free) e 756 aplicativos pagos (Paid)

![type_distinct](img/type_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT type, COUNT(*)`
`FROM apps_data`
`GROUP BY type`

## Coluna 'Price'

Dicionário: coluna 'Price' contém preço do aplicativo, sendo 0 = Grátis

Valores iniciais

![price_distinct](img/price_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT price, COUNT(*)`
`FROM apps_data`
`GROUP BY price`

Como essa coluna pode ser usada para cálculos, foi necessário retirar o '$' antes do valor

![price_update_cifrao](img/price_update_cifrao.PNG)

**COMANDO USADO ACIMA:**
`UPDATE apps_data`
`SET price = LTRIM(price, '$')::NUMERIC`
`WHERE price LIKE '%$%'`

E alterei o tipo para numérico

![price_alter_type](img/price_alter_type.PNG)

**COMANDO USADO ACIMA:**
`ALTER TABLE apps_data`
`ALTER COLUMN price TYPE NUMERIC`
`USING price::NUMERIC`

Podendo assim realizar operações matemáticas como abaixo

![price_min_max](img/price_min_max.PNG)

**COMANDO USADO ACIMA:**
`SELECT MIN(price), MAX(price)`
`FROM apps_data`

## Coluna 'Content_rating'

Dicionário: coluna 'Content_rating' contém a faixa etária atribuída ao aplicativo

Essa coluna não apresenta valores faltantes, e sim 6 categorias de faixa etária

![content_rating_distinct](img/content_rating_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT content_rating`
`FROM apps_data`
`ORDER BY content_rating DESC`

## Coluna 'Genres'

Dicionário: coluna 'Genres' contém os gêneros que o aplicativo possui

Como 'Genres' é do tipo VARCHAR, não foi necessário tratar o único valor faltante representado por 'NULL'.

Podemos ver que são 118 gêneros distintos

![genre_distinct](img/genre_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT genres, COUNT(*)`
`FROM apps_data`
`GROUP BY genres`
`ORDER BY genres DESC`


## Coluna 'Last_update'

Dicionário: coluna 'Last_update' contém a data que o aplicativo foi atualizado mais recente

Por ser uma coluna do tipo DATE e não ter valores faltantes, só precisei verificar o intervalo de datas que essa coluna abrange

![last_update_min_max](img/last_update_min_max.PNG)

**COMANDO USADO ACIMA:**
`SELECT MIN(last_update), MAX(last_update)`
`FROM apps_data`

## Coluna 'Current_ver'

Dicionário: coluna 'Current_ver' contém a versão atual do aplicativo 

Como 'Current_ver' é do tipo VARCHAR, não foi necessário tratar o único valor faltante representado por 'NULL'.

Podemos ver que são 2817 versões distintas

![current_ver_distinct](img/current_ver_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT current_ver, COUNT(*)`
`FROM apps_data`
`GROUP BY current_ver`
`ORDER BY current_ver DESC`

## Coluna 'Android_ver'

Dicionário: coluna 'Android_ver' contém a versão do Android necessária para o aplicativo funcionar

Como 'Android_ver' é do tipo VARCHAR, não foi necessário tratar os 2 valores faltantes representados por 'NaN', nem os valores representados por 'Varies with device'.

Podemos ver que são 32 versões distintas

![android_ver_distinct](img/android_ver_distinct.PNG)

**COMANDO USADO ACIMA:**
`SELECT DISTINCT android_ver, COUNT(*)`
`FROM apps_data`
`GROUP BY android_ver`
`ORDER BY android_ver DESC`

# Resultados após o tratamento e limpeza dos dados

## App - nome do aplicativo

    Tabela agora possui 9660 registros sem valores duplicados.
    
## Category - categoria

    33 categorias distintas e 1 NULL.
    
## Rating - nota

    Valores variam de 1.0 a 5.0. Foi atribuído o valor 0 para os campos com 'NaN'.

## Reviews - avaliação

    Valores variam de 0 a 78128208.

## Size - tamanho

    Valores variam de 8,5 KB a 100000 KB (ou 100 MB).
    Valor 0 representa 'Varies with device' (varia com dispositivo).

## Installs - quantidade de instalações

    São 20 tipos distintos, com valores variando de 0 a 1000000000 (1 Bilhão).

## Type - tipo

    8903 valores Free (grátis) / 756 Paid (pagos) / 1 NaN (sem valor).

## Price - preço

    Valores variam de $0,00 a $400,00.

## Content_rating - faixa etária

    Contém 6 categorias distintas.

## Genres - gêneros
    
    Contém 118 categorias distintas, e 1 valor NULL
    Podem ser agrupados como 'Trivia;Education'

## Last_update - data da última atualização

    Variam de 21/05/2010 a 08/08/2018

## Current_ver - versão atual
    
    Contém 2817 categorias distintas, e 1 valor NULL

## Android_ver - versão do Android

    Contém 32 categorias distintas.
    Mais de 991 registros na categoria 'Varies with Device' e 2 registros como 'NaN'

# Análise gráfica de alguns dados

## Category (Categoria)

Através desse gráfico, podemos ver que as categorias 'Família', 'Jogo' e 'Ferramentas' se destacam sob as demais, sendo a 'Família' com quase o dobro que a segunda colocada. As 3 juntas representam mais da metade de todas as categorias juntas. 

![top10_categoria](img/top10_categoria.PNG)


## Type (Tipo)

Com esse gráfico, fica claro que os apps 'Grátis' são os mais numerosos na Play Store, representando mais de 90% do total.

![Tipo](img/Tipo.PNG)


## Content_rating (Faixa etária)

Temos 6 faixas distintas, sendo a primeira 'Para todos' a que contém maior número de aplicativos, o que nos leva a refletir que um app que se restringe a uma faixa etária específica não atrai todos os consumidores, claramente. 

Talvez seja uma falha da Play Store em como avaliar a qual faixa o aplicativo deve ser colocado, provavelmente quem define isso é o próprio criador, que não vai limitar o seu próprio produto.

![content_rating](img/content_rating.PNG)


## Genres (Gêneros)

Lembrando que são mais de 100 gêneros diferentes, e muitos deles estão agrupados (ex: Education;Sport), o que dificulta o cálculo de quantos gêneros distintos realmente existem.

Mas podemos ver que os que têm maiores quantidades selecionaram somente um gênero simples e não um composto.

'Ferramentas', 'Entretenimento' e 'Educação' formam o Top 3, sendo 'Ferramentas' o maior e os demais estão bem pareados.

![genres](img/genres.PNG)


## Rating (Nota do app)

No gráfico abaixo, pode se ver a distribuição do número de instalações que o aplicativo teve em função da nota que possui.

![installsxrating](img/installsxrating.PNG)

Selecionando os 10 maiores em número de instalações, vemos que eles variam de 3,80 a 4,70. Pode acabar gerando a questão: 'Por que 4,80 pra cima não estão nessa visualização?'

Pode ser uma dúvida ao olharmos pro gráfico, mas pra quem usa algum aplicativo que tem o sistema de notas, sabe que de 4,80 pra cima, normalmente são os aplicativos novos ou com poucos usuários. Essa variação do gráfico nos mostra que uma vez que o aplicativo é baixado tantas vezes, sua nota acaba se estabilizando entre 4,30 e 4,50.

![top10_installsxrating](img/top10_installsxrating.PNG)

No gráfico abaixo, pode se ver a distribuição do total que os aplicativos faturaram em função da nota que possuem.

![rating_soma_price](img/rating_soma_price.PNG)

Os aplicativos com notas 3,80 foram os que mais faturaram, como vimos no gráfico anterior (instalações por nota), que os aplicativos com notas 3,80 foram os únicos abaixo dos de nota 4,00 e melhor do que os de notas 4,80 pra cima.

Apesar da nota ser 'baixa' comparada aos demais, eles tiveram um número de instalações muito grandes (vide top 10 acima), e que com o preço médio deles, os tornaram líder do gráfico abaixo.

Diferente do 4,30 qe foi líder do anterior, e agora é vice-líder nesse. Provavelmente por ter um preço médio inferior, ou simplesmente menos aplicativos com essa nota.

![soma_price_rating](img/soma_price_rating.PNG)


## Price

![top10_installsxprice](img/top10_installsxprice.PNG)

![top10_installsxprice_paid](img/top10_installsxprice_paid.PNG)




## Content_rating

![content_rating_x_installs](img/content_rating_x_installs.PNG)

![content_rating_x_price](img/content_rating_x_price.PNG)

![content_rating_x_category](img/content_rating_x_category.PNG)


## Category

![soma_installs_X_category](img/soma_installs_X_category.PNG)

![media_installs_x_category](img/media_installs_x_category.PNG)

![media_size_x_category](img/media_size_x_category.PNG)

![top10_media_size_x_category](img/top10_media_size_x_category.PNG)

![media_price_x_category](img/media_price_x_category.PNG)

![top10_media_reviews_X_category](img/top10_media_reviews_X_category.PNG)
